# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [2]:
# your code here
import pandas as pd
import numpy as np

data = pd.read_csv("/Users/juliettegoardon/Desktop/lab-deep-learning/your-code/tic-tac-toe.csv")
data.head()

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   TL      958 non-null    object
 1   TM      958 non-null    object
 2   TR      958 non-null    object
 3   ML      958 non-null    object
 4   MM      958 non-null    object
 5   MR      958 non-null    object
 6   BL      958 non-null    object
 7   BM      958 non-null    object
 8   BR      958 non-null    object
 9   class   958 non-null    bool  
dtypes: bool(1), object(9)
memory usage: 68.4+ KB


In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
for column in data.columns:
    data[column] = label_encoder.fit_transform(data[column])

print(data)

     TL  TM  TR  ML  MM  MR  BL  BM  BR  class
0     2   2   2   2   1   1   2   1   1      1
1     2   2   2   2   1   1   1   2   1      1
2     2   2   2   2   1   1   1   1   2      1
3     2   2   2   2   1   1   1   0   0      1
4     2   2   2   2   1   1   0   1   0      1
..   ..  ..  ..  ..  ..  ..  ..  ..  ..    ...
953   1   2   2   2   1   1   1   2   2      0
954   1   2   1   2   2   1   2   1   2      0
955   1   2   1   2   1   2   2   1   2      0
956   1   2   1   1   2   2   2   1   2      0
957   1   1   2   2   2   1   1   2   2      0

[958 rows x 10 columns]


In [5]:
# Separate inputs (features) and output (target)
X = data.drop('class', axis=1)  
y = data['class']  

print("Inputs (Features):")
print(X)
print("\nOutput (Target):")
print(y)


Inputs (Features):
     TL  TM  TR  ML  MM  MR  BL  BM  BR
0     2   2   2   2   1   1   2   1   1
1     2   2   2   2   1   1   1   2   1
2     2   2   2   2   1   1   1   1   2
3     2   2   2   2   1   1   1   0   0
4     2   2   2   2   1   1   0   1   0
..   ..  ..  ..  ..  ..  ..  ..  ..  ..
953   1   2   2   2   1   1   1   2   2
954   1   2   1   2   2   1   2   1   2
955   1   2   1   2   1   2   2   1   2
956   1   2   1   1   2   2   2   1   2
957   1   1   2   2   2   1   1   2   2

[958 rows x 9 columns]

Output (Target):
0      1
1      1
2      1
3      1
4      1
      ..
953    0
954    0
955    0
956    0
957    0
Name: class, Length: 958, dtype: int64


In [6]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

X_normalized = scaler.fit_transform(X)

X_normalized_df = pd.DataFrame(X_normalized, columns=X.columns)

print("Normalized Input Data:")
print(X_normalized_df)


Normalized Input Data:
      TL   TM   TR   ML   MM   MR   BL   BM   BR
0    1.0  1.0  1.0  1.0  0.5  0.5  1.0  0.5  0.5
1    1.0  1.0  1.0  1.0  0.5  0.5  0.5  1.0  0.5
2    1.0  1.0  1.0  1.0  0.5  0.5  0.5  0.5  1.0
3    1.0  1.0  1.0  1.0  0.5  0.5  0.5  0.0  0.0
4    1.0  1.0  1.0  1.0  0.5  0.5  0.0  0.5  0.0
..   ...  ...  ...  ...  ...  ...  ...  ...  ...
953  0.5  1.0  1.0  1.0  0.5  0.5  0.5  1.0  1.0
954  0.5  1.0  0.5  1.0  1.0  0.5  1.0  0.5  1.0
955  0.5  1.0  0.5  1.0  0.5  1.0  1.0  0.5  1.0
956  0.5  1.0  0.5  0.5  1.0  1.0  1.0  0.5  1.0
957  0.5  0.5  1.0  1.0  1.0  0.5  0.5  1.0  1.0

[958 rows x 9 columns]


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [10]:
# your code here

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

data = pd.read_csv("/Users/juliettegoardon/Desktop/lab-deep-learning/your-code/tic-tac-toe.csv")

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
for column in data.columns:
    data[column] = label_encoder.fit_transform(data[column])

X = data.drop('class', axis=1)  
y = data['class'] 

scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(X)

print("Normalized Input Data:")
print(X_normalized)

X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

model = tensorflow.keras.Sequential()

model.add(tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(2, activation='softmax'))  # Using softmax for multi-class classification

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy}")

model.save("tic-tac-toe.model")


Normalized Input Data:
[[1.  1.  1.  ... 1.  0.5 0.5]
 [1.  1.  1.  ... 0.5 1.  0.5]
 [1.  1.  1.  ... 0.5 0.5 1. ]
 ...
 [0.5 1.  0.5 ... 1.  0.5 1. ]
 [0.5 1.  0.5 ... 1.  0.5 1. ]
 [0.5 0.5 1.  ... 0.5 1.  1. ]]


NameError: name 'tensorflow' is not defined

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [ ]:
# your code here

import numpy as np
import tensorflow as tf

model = tf.keras.models.load_model("tic-tac-toe.model")

random_indices = np.random.choice(len(X_test), size=5, replace=False)

X_sample = X_test[random_indices]
y_sample = y_test[random_indices]

predictions = model.predict(X_sample)

predicted_classes = np.argmax(predictions, axis=1)

print("True Labels:", y_sample)
print("Predicted Labels:", predicted_classes)


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [ ]:
# your code here

import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

model = tf.keras.Sequential()

model.add(tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(32, activation='relu'))  # Adding more layers
model.add(tf.keras.layers.Dense(2, activation='softmax'))

custom_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)  # Adjust learning rate here
model.compile(optimizer=custom_optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=20, batch_size=64, validation_split=0.2)  # Adjust epochs here

test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy}")

model.save("tic-tac-toe_improved.model")

**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
# your answer here